<a href="https://colab.research.google.com/github/ShaelinN/VAE_RECOMMENDER_IMPLICIT_FEEDBACK/blob/main/VAE_V3_secondary_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports and Dirs

In [1]:
#!pip3 install tensorflow-ranking
#!pip3 install tensorflow
#!pip3 install numpy
#!pip3 install pickle
#!pip3 install math

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as tfback
from tensorflow.keras import layers, activations, Model, losses
from tensorflow.keras.callbacks import Callback, ModelCheckpoint
#import tensorflow_ranking as tfr

import numpy as np
import os
import sys
import pickle
import math
import argparse

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
"""
#uncomment if running as .py on cluster such as HIPPO
#comment if running as .ipynb on Google Colab

parser = argparse.ArgumentParser()
parser.add_argument('--root', type=str)
parser.add_argument('--input_data', type=str)
parser.add_argument('--training_results', type=str, default="training_results")

parser.add_argument('--prim_pel', type=str, default="prim_pel.txt")
parser.add_argument('--prim_pbl', type=str, default="prim_pbl.txt")
parser.add_argument('--prim_pevl', type=str, default="prim_pevl.txt")

parser.add_argument('--sec_pel', type=str, default="sec_pel.txt")
parser.add_argument('--sec_pbl', type=str, default="sec_pbl.txt")
parser.add_argument('--sec_pevl', type=str, default="sec_pevl.txt")


parser.add_argument('--weights', type=str)
parser.add_argument('--intermediatedim', type=int)
parser.add_argument('--epochs', type=int)
parser.add_argument('--batchsize', type=int)
parser.add_argument('--klannealrate', type=float)
args = parser.parse_args()
"""

In [13]:
#uncomment if running as .ipynb on Google Colabas
#comment if running as  .py on cluster such as HIPPO
class argclass(object):
  def __init__(self):
    self.root = "/content/drive/MyDrive/COMP700_Honours Project"
    self.input_data = "Data/remove_low_interaction_users/split/matrices/implicit"
    self.training_results ="training_results"

    self.prim_pel = "prim_pel.txt"
    self.prim_pbl = "prim_pbl.txt"
    self.prim_pevl = "prim_pevl.txt"

    self.sec_pel = "sec_pel.txt"
    self.sec_pbl = "sec_pbl.txt"
    self.sec_pevl = "sec_pevl.txt"

    self.weights = None #"vae_epoch_01_loss_210.97.hdf5"
    self.intermediatedim = 512 
    self.epochs = 10
    self.batchsize = 200
    self.klannealrate = 0.001
args = argclass()


In [14]:
root = args.root

input_data = os.path.join(root, args.input_data)

training_results = os.path.join(root, args.training_results)


try:
  os.mkdir(training_results)
exceptFileExistsError:
  pass

weights = args.weights
if weights is None:
  sys.stderr.write("Weights cannot be None. Perform primary training and load a resultant 'filename.hdf5' weights file as an arg")
  exit(-1)
weights = os.path.join(training_results, weights)
"""
try:
  weights = os.path.join(training_results, weights)
except TypeError:
  pass
"""

#annealing_rate = args.klannealrate if not args.klannealrate is None else 0.0001
#intermediate_dim = args.intermediatedim if args.intermediatedim is not None else 512
#total_num_epochs = args.epochs if args.epochs is not None else 5
#batch_size = args.batchsize if args.batchsize is not None else 400



print(root)
print(input_data)
print(training_results)
print(weights)

/content/drive/MyDrive/COMP700_Honours Project
/content/drive/MyDrive/COMP700_Honours Project/Data/remove_low_interaction_users/split/matrices/implicit
/content/drive/MyDrive/COMP700_Honours Project/training_results
/content/drive/MyDrive/COMP700_Honours Project/training_results/lie


#Model design
* same as primary
(minor change: higher annealing rate)

In [ ]:
class Sampling(layers.Layer):
  def __init__(self, name="Sampling", **kwargs):
    super(Sampling, self).__init__(name=name, **kwargs)
    
  def call(self, inputs):
    z_mean, z_log_var = inputs
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]


    #epsilon = distribution.sample()
    
    epsilon = tfback.random_normal(shape=(batch,dim))
    sample = epsilon * tf.exp(0.5 * z_log_var)  +   z_mean  #Reparametrization trick: convert from standard normal to desired distribution

    return sample

In [ ]:
class BatchHistory(keras.callbacks.Callback):  
  def __init__(self, pel, pbl, pevl):
    super(BatchHistory,self).__init__() 
    self.loss = [] 
    self.val_loss = 0.0
    self.pel = pel
    self.pbl = pbl
    self.pevl = pevl

  def on_train_batch_end(self, batch, logs=None):   
    self.loss.append(logs.get('loss'))


  def on_epoch_end(self, epoch, logs=None):
    self.val_loss = logs.get('val_loss')
    self.loss = [str(i) for i in self.loss]

    with open(os.path.join(training_results,pevl), "a") as pe_v_loss:
      pe_v_loss.write('\n'+ str(self.val_loss))

    with open(os.path.join(training_results,pbl), "a") as pb_loss:
      pb_loss.write("\n")
      pb_loss.writelines('\n'.join(self.loss))

    with open(os.path.join(training_results,pel), "a") as pe_loss:
      pe_loss.write('\n'+ str(self.loss[len(self.loss)-1]))
    self.loss = []
    

In [ ]:
#original_dim = 127350
annealing_rate = args.klannealrate if not args.klannealrate is None else 0.001 #10x faster annealing in expectation of less training data. should it be so? is annealing even required in this phase?
KLBeta = 0
def VAE_loss(y_true, y_pred):
    global KLBeta
    reconst_loss =  original_dim * losses.binary_crossentropy(y_true, y_pred)
    KLDiv = KLBeta * losses.kl_divergence(y_true, y_pred)

    KLBeta = min(KLBeta+annealing_rate , 1) #update weight of KL factor

    #return (KLDiv,reconst_loss)

    return  reconst_loss + KLDiv


In [ ]:
class vae_builder(object):
  def __init__(self, original_dim, intermediate_dim, latent_dim, name='VAE'):
    self.name = name
    self.original_dim = original_dim
    self.intermediate_dim = intermediate_dim
    self.latent_dim = latent_dim

  
  def build(self):
    self.input = layers.Input(self.original_dim, name = 'input')
    self.dropout = layers.Dropout(rate=0.5)(self.input)

    #encoder
    self.e1 = layers.Dense(self.intermediate_dim, activation=activations.hard_sigmoid, name = 'e1')(self.dropout)
    self.e2 = layers.Dense(self.intermediate_dim, activation=activations.hard_sigmoid, name = 'e2')(self.e1)
    self.e3 = layers.Dense(self.intermediate_dim, activation=activations.hard_sigmoid, name = 'e3')(self.e2)
    self.e4 = layers.Dense(self.intermediate_dim, activation=activations.hard_sigmoid, name = 'e4')(self.e3)
    self.e5 = layers.Dense(self.intermediate_dim, activation=activations.hard_sigmoid, name = 'e5')(self.e4)

    #sampling
    self.mean = layers.Dense(self.latent_dim, name = 'mean')(self.e5)  
    self.log_var = layers.Dense(self.latent_dim, name = 'log_var')(self.e5)  
    self.sampling = Sampling()([self.mean, self.log_var])

    #decoder
    self.d1 = layers.Dense(self.intermediate_dim, activation='relu',name = 'd1')(self.sampling)
    self.d2 = layers.Dense(self.original_dim, activation='relu', name = 'd2')(self.d1)  
    
    self.output = layers.Activation(activations.swish, name = 'output')(self.d2)

    #to model
    vae = Model(inputs = self.input, outputs = self.output, name = self.name)
    return vae

In [ ]:
class datagen(keras.utils.Sequence):
  def __init__(self, x_set, batch_size, max_samples_per_epoch=None):
    self.x = x_set
    self.batch_size = batch_size

    self.shuffled_idx = np.arange(np.shape(self.x)[0]) #get indexes
    np.random.shuffle(self.shuffled_idx) # shuffle

    if max_samples_per_epoch is not None:
      self.shuffled_idx = self.shuffled_idx[:max_samples_per_epoch] #cutoff at max no of samples allowed in epoch

  def on_epoch_end(self):
    self.shuffled_idx = np.arange(np.shape(self.x)[0]) #get indexes
    np.random.shuffle(self.shuffled_idx) # shuffle

  def __len__(self):
      #return math.ceil((self.x.shape[0]) / self.batch_size)
      return len(self.shuffled_idx)
      
  def __getitem__(self, idx):
    b_idx = idx * self.batch_size
    e_idx = (idx + 1) * self.batch_size
    idx = self.shuffled_idx[b_idx:e_idx] #cut slice of indexes using begin and end indexes

    batch_x = np.array(self.x[idx].todense())

    batch_y = np.array(self.x[idx].todense())


    return batch_x , batch_y


#Secondary Training
##reminder:
* Train on train to replicaate train (with fuzziness i.e. new recommendations)

* train on test_train to replicate test_train (with fuzziness i.e. new recommendations i.e hopefully something that has stuff in common with test_test)

* test on test_train to see if the fuzziness is actually what is in test_test

* eval on test_test to compare the fuzziness around output of test_train to the true vals of test_test

In [ ]:
test_train = pickle.load( 
    open(os.path.join(input_data,"test_training_impl.pkl") , 'rb')
)
vad = pickle.load( 
    open(os.path.join(input_data,"vad_impl.pkl") , 'rb')
)

total_num_samples = test_train.shape[0]
original_dim = test_train.shape[1]
intermediate_dim = args.intermediatedim if args.intermediatedim is not None else 512
latent_dim = intermediate_dim//2 #64
total_num_epochs = 2 #args.epochs if args.epochs is not None else 5
batch_size = args.batchsize if args.batchsize is not None else 400


test_train_gen = datagen(test_train , batch_size)
vad_gen = datagen(vad ,  batch_size)

In [ ]:
#REFRESH CALLBACKS BEFORE EACH RUN
#or else separate runs in same session will behave oddly
checkpoint = ModelCheckpoint(filepath=os.path.join(training_results,"vae_final_epoch_{epoch:02d}_loss_{loss:.2f}.hdf5"), verbose=1, save_best_only=False)
batch_hist = BatchHistory(args.sec_pel, args.sec_pbl, args.sec_pevl)

#BUILD STRUCTURE
v = vae_builder(original_dim, intermediate_dim, latent_dim).build()

#configurable optimiser
slow_adam = keras.optimizers.Adam(learning_rate=1e-4)
'''
metrics = [
           tfr.keras.metrics.NDCGMetric(topn=20),
           tfr.keras.metrics.RecallMetric(topn=20)
           ]
'''
v.compile(optimizer=slow_adam, loss=VAE_loss)#, metrics = metrics)

#PICK UP FROM WHERE LEFT OFF
remaining_epochs = total_num_epochs # no start from crash since only small amount of data in secondary train
v.load_weights(weights)

#RUN
print("commisioned to run for",total_num_epochs,"epochs total")
print("will run for", remaining_epochs, "more epochs")

v.fit(test_train_gen, verbose=True, epochs=remaining_epochs, validation_data=vad_gen, callbacks=[checkpoint,batch_hist])

print("complete")

In [ ]:
#keras.utils.plot_model(v, show_shapes=True, to_file=os.path.join(root, "vae.png"))